# Sudoku: Integer Programming

Defining Sudoku as an integer programming problem involves formulating constraints to ensure that each row, column, and 3x3 subgrid contains the numbers 1 through 9 exactly once. Let’s set up this formulation step-by-step.

### 1. Define Variables

Let $x_{i,j,k}$ be a binary variable where:

$\quad x_{i,j,k} = 
\begin{cases} 
1 & \text{if cell } (i,j) \text{ contains the number } k \\
0 & \text{otherwise}
\end{cases}$

Here, $i, j \in {1,..,9} $ represents the possible numbers in each cell and $ k \in {1,...,9} $ represents the possible numbers in each cell.

### 2. Objective Function
In Sudoku, our goal is to fill the grid according to the rules, not to maximize or minimize a specific objective. So, there is no objective function, and it becomes a feasibility problem.

### 3. Constraints
To define a valid Sudoku solution, we impose the following constraints:



#### a. Cell Constraints
Each cell $(i,j)$ must contain exactly one number $k$ from 1 to 9:

$\sum_{k=1}^{9} x_{i,j,k} = 1 \quad \forall \, i, j \in \{1, \ldots, 9\}$

#### b. Row Constraints 
Each number $k$ appears exactly once in each row $i$:

$\sum_{j=1}^{9} x_{i,j,k} = 1 \quad \forall \, i, k \in \{1, \ldots, 9\}$

#### c. Column Constraints
Each number $k$ appears exactly once in each column $j$:

$\sum_{i=1}^{9} x_{i,j,k} = 1 \quad \forall \, j, k \in \{1, \ldots, 9\}$

#### d. Subgrid Constraints
Each number $k$ appears exactly once in each 3x3 subgrid. Let $I$ and $J$ represent the starting row and column indices of each 3x3 subgrid (e.g., for the top-left subgrid, $
I,J \in {1,4,7}$). Then:

$\sum_{i=I}^{I+2} \sum_{j=J}^{J+2} x_{i,j,k} = 1 \quad \forall \, k \in \{1, \ldots, 9\}, \, I, J \in \{1, 4, 7\}$

#### e. Initial Conditions (pre-filled cells):
For cells with given numbers in the initial Sudoku puzzle, we add fixed constraints. If cell $(i,j)$ is initially filled with the number $k$ then:

$x_{i,j,k} = 1$

### Summary
This formulation turns Sudoku into a system of equations and inequalities that, when satisfied, yields a correct Sudoku solution.

In [2]:
import cvxpy as cp
import numpy as np

# Sudoku grid size
n = 9  # Grid size for 9x9 Sudoku
num_cells = n * n  # Total cells in the grid

# Define a 2D binary variable where x[cell, k] = 1 if the cell contains number k+1
x = cp.Variable((num_cells, n), boolean=True)

# Helper function to get the cell index from row and column
def cell_index(i, j):
    return i * n + j

# Constraints
constraints = []

# 1. Cell Constraints: Each cell contains exactly one number
for cell in range(num_cells):
    constraints.append(cp.sum(x[cell, :]) == 1)

# 2. Row Constraints: Each number appears exactly once in each row
for i in range(n):
    for k in range(n):
        # Convert the generator to a list to avoid TypeError
        constraints.append(cp.sum([x[cell_index(i, j), k] for j in range(n)]) == 1)

# 3. Column Constraints: Each number appears exactly once in each column
for j in range(n):
    for k in range(n):
        # Convert the generator to a list to avoid TypeError
        constraints.append(cp.sum([x[cell_index(i, j), k] for i in range(n)]) == 1)

# 4. Subgrid Constraints: Each number appears exactly once in each 3x3 subgrid
for I in range(0, n, 3):
    for J in range(0, n, 3):
        for k in range(n):
            # Convert the generator to a list to avoid TypeError
            constraints.append(cp.sum([x[cell_index(I + di, J + dj), k] 
                                       for di in range(3) for dj in range(3)]) == 1)

# 5. Initial Conditions (Pre-filled cells in the Sudoku puzzle)
# Example of a partially-filled Sudoku puzzle grid
initial_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Apply constraints for the given numbers in the initial grid
for i in range(n):
    for j in range(n):
        if initial_grid[i][j] != 0:
            # Set x[cell_index(i, j), k] = 1 for the specific given number k in cell (i, j)
            k = initial_grid[i][j] - 1  # Convert value to zero-based index
            constraints.append(x[cell_index(i, j), k] == 1)

# Objective function: No objective since this is a feasibility problem
objective = cp.Maximize(0)

# Define the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve(solver=cp.CBC)  # Use CBC solver

# Extract and print solution if feasible
if problem.status == cp.OPTIMAL:
    solution = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if x[cell_index(i, j), k].value > 0.5:  # Check if x[cell_index(i, j), k] is approximately 1
                    solution[i, j] = k + 1  # Convert to 1-based index
    print("Solved Sudoku grid:")
    print(solution)
else:
    print("No solution found.")


Solved Sudoku grid:
[[5 3 4 6 7 8 9 1 2]
 [6 7 2 1 9 5 3 4 8]
 [1 9 8 3 4 2 5 6 7]
 [8 5 9 7 6 1 4 2 3]
 [4 2 6 8 5 3 7 9 1]
 [7 1 3 9 2 4 8 5 6]
 [9 6 1 5 3 7 2 8 4]
 [2 8 7 4 1 9 6 3 5]
 [3 4 5 2 8 6 1 7 9]]
